# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 150  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 2)                 34 

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/150
3122/3122 - 6s - loss: 0.5787 - val_loss: 0.3643 - 6s/epoch - 2ms/step
Epoch 2/150
3122/3122 - 3s - loss: 0.3101 - val_loss: 0.2757 - 3s/epoch - 1ms/step
Epoch 3/150
3122/3122 - 5s - loss: 0.2491 - val_loss: 0.2289 - 5s/epoch - 1ms/step
Epoch 4/150
3122/3122 - 3s - loss: 0.2102 - val_loss: 0.1964 - 3s/epoch - 838us/step
Epoch 5/150
3122/3122 - 3s - loss: 0.1823 - val_loss: 0.1727 - 3s/epoch - 806us/step
Epoch 6/150
3122/3122 - 3s - loss: 0.1630 - val_loss: 0.1565 - 3s/epoch - 811us/step
Epoch 7/150
3122/3122 - 2s - loss: 0.1494 - val_loss: 0.1436 - 2s/epoch - 800us/step
Epoch 8/150
3122/3122 - 4s - loss: 0.1394 - val_loss: 0.1374 - 4s/epoch - 1ms/step
Epoch 9/150
3122/3122 - 3s - loss: 0.1321 - val_loss: 0.1350 - 3s/epoch - 881us/step
Epoch 10/150
3122/3122 - 3s - loss: 0.1263 - val_loss: 0.1254 - 3s/epoch - 1ms/step
Epoch 11/150
3122/3122 - 3s - loss: 0.1218 - val_loss: 0.1201 - 3s/epoch - 811us/step
Epoch 12/150
3122/3122 - 3s - loss: 0.1176 - val_loss: 0.1189 - 3s/epoch 

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [6]:
fit = np.load("data/fit.npz")

In [7]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)